Course Instructor: Bernd Neumayr, JKU

# UE05: SHACL

Complete the **10 tasks (1 point per task)** in the `4. SHACL` sheet of `SemAI.jar` first and then transfer them to this notebook.

For each task include:
- A headline including the task number
- The task description 
- The data graph and your solution (the shapes graph) in executable form
- After executing the validation, print out the validation results in tabular form.  


In [1]:
# Install required packages in the current Jupyter kernel
#!pip install -q rdflib
#!pip3 install -q pyshacl

In [2]:
# Imports
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace, Dataset
from rdflib.namespace import FOAF , XSD , RDFS 
from rdflib.plugins.sparql.processor import SPARQLResult
from rdflib.namespace import NamespaceManager

from pyshacl import validate

import pandas as pd

def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = [ { var : res[var].n3(graph.namespace_manager) if (isinstance(res[var],URIRef) and use_prefixes) else res[var] for var in results.vars } for res in results ]     
                                        # construct a list of dictionaries, as intermediate format to construct the pandas DataFrame, use prefixes to abbreviate URIs                
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def validation_report_as_dataframe(validation_report):
  df = sparql_select(results_graph,"""
		SELECT  ?focusNode ?resultPath ?value ?sourceConstraintComponent ?sourceShape ?resultMessage
		WHERE
  		{ ?vr	a sh:ValidationResult ;
						sh:focusNode ?focusNode ;
						sh:sourceConstraintComponent ?sourceConstraintComponent ;
						sh:sourceShape ?sourceShape ;
						sh:resultMessage ?resultMessage .					 
				OPTIONAL { ?vr sh:value ?value . }
				OPTIONAL { ?vr sh:resultPath ?resultPath . }
  		}
  """,use_prefixes=True)
  return df

def shacl_validate(dg,sg):
  return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)  
  

def shacl_validate_with_rules(dg,sg):
	return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=True,
      iterate_rules=True, inplace=True,
      js=False,
      debug=False)

## Task 1
Wenn eine Person eine x:knows Beziehung hat, dann sollte das Ziel dieser Beziehung als IRI angegeben sein.

In [3]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Peter>  rdf:type  <Person> ;
        :knows    <John> ;
        :knows    [ :name  "Peter" ] .

<John>  :knows  [ :name   "Mary" ] .     
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<PersonShape> a sh:NodeShape ;
    sh:targetClass <Person> ;
    sh:property <PersonKnowsShape> .

<PersonKnowsShape> a sh:PropertyShape ;
    sh:path :knows ;
    sh:nodeKind sh:IRI .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Peter ;
            sh:resultMessage "Value is not of Node Kind sh:IRI" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeKindConstraintComponent ;
            sh:sourceShape :PersonKnowsShape ;
            sh:value [ a rdfs:Resource ;
                    :name "Peter" ] ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Peter,:knows,n038d5721df66484faf8c5552c2c5d084b1,sh:NodeKindConstraintComponent,:PersonKnowsShape,Value is not of Node Kind sh:IRI


## Task 2
Die Klassen Man und Woman sind disjunkt. Hinweis: verwenden Sie sh:not.

In [4]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<YoungMan>  rdfs:subClassOf  <Man> .

<John>  rdf:type  <YoungWoman> , <YoungMan> .

<Mary>  rdf:type  <Woman> .

<Peter>  rdf:type  <Man> .

<YoungWoman>  rdfs:subClassOf  <Woman> .

<Jane>  rdf:type  <YoungWoman> .

<Susi>  rdf:type  <YoungWoman> , <Man> .   
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<ManShape> a sh:NodeShape ;
    sh:targetClass <Man> ;
    sh:not [sh:class <Woman>] .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :John ;
            sh:resultMessage "Node :John conforms to shape [ sh:class :Woman ]" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NotConstraintComponent ;
            sh:sourceShape :ManShape ;
            sh:value :John ],
        [ a sh:ValidationResult ;
            sh:focusNode :Susi ;
            sh:resultMessage "Node :Susi conforms to shape [ sh:class :Woman ]" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NotConstraintComponent ;
            sh:sourceShape :ManShape ;
            sh:value :Susi ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:John,None,:John,sh:NotConstraintComponent,:ManShape,Node :John conforms to shape [ sh:class :Woman ]
1,:Susi,None,:Susi,sh:NotConstraintComponent,:ManShape,Node :Susi conforms to shape [ sh:class :Woman ]


## Task 3
Wenn eine Person eine :knows Beziehung hat, dann muss das Ziel dieser Beziehung eine Person sein.

In [5]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Peter>  rdf:type  <Person> ;
        :knows    <Mary> ;
        :knows    [ :name  "Mary" ] ;
        :knows    [ rdf:type  <Person> ] .

<Jane>  :knows  <Jim> . 
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<PersonShape> a sh:NodeShape ;
    sh:targetClass <Person> ;
    sh:property <PersonKnowsShape> .

<PersonKnowsShape>
    sh:path :knows ;
    sh:class <Person> .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Peter ;
            sh:resultMessage "Value does not have class :Person" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :PersonKnowsShape ;
            sh:value [ a rdfs:Resource ;
                    :name "Mary" ] ],
        [ a sh:ValidationResult ;
            sh:focusNode :Peter ;
            sh:resultMessage "Value does not have class :Person" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :PersonKnowsShape ;
            

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Peter,:knows,:Mary,sh:ClassConstraintComponent,:PersonKnowsShape,Value does not have class :Person
1,:Peter,:knows,n7179371faf384229b9a26b99a0d6cfe6b1,sh:ClassConstraintComponent,:PersonKnowsShape,Value does not have class :Person


## Task 4
Jede Person hat genau ein Alter (:age). Eine Person ist mindestens 0 und maximal 150 Jahre alt. Erwachsene (Klasse Adult) sind mindestens 19 Jahre alt. Senioren sind mindestens 65 Jahre alt. Kinder sind maximal 12 Jahre alt.
Verwenden Sie sh:minInclusive und sh:maxInclusive (siehe SHACL Recommendation).

In [6]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<ABC>   :age    -1 , 27 , 197 .

<John>  rdf:type  <Person> ;
        :age      151 .

<Adult>  rdfs:subClassOf  <Person> .

<Beny>  rdf:type  <Child> ;
        :age      13 .

<Zoe>   rdf:type  <Child> ;
        :age      12 .

<Mary>  rdf:type  <Woman> ;
        :age      17 .

<Bibi>  rdf:type  <Person> ;
        :age      0 .

<Peter>  rdf:type  <Person> .

<Senior>  rdfs:subClassOf  <Adult> .

<Jane>  rdf:type  <Woman> ;
        :age      67 .

<Lili>  rdf:type  <Person> ;
        :age      -1 .

<Senior1>  rdf:type  <Senior> ;
        :age      65 .

<Child>  rdfs:subClassOf  <Person> .

<Bibo>  rdf:type  <Person> ;
        :age      34 , 12 .

<Woman>  rdfs:subClassOf  <Adult> .

<Senior2>  rdf:type  <Senior> ;
        :age      64 . 
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<PersonShape> a sh:NodeShape ;
    sh:targetClass <Person> ;
    sh:property <PersonAgeShape> .

<PersonAgeShape>
    sh:path :age;
    sh:minInclusive 0 ;
    sh:maxCount 1 ;
    sh:minCount 1 ;
    sh:maxInclusive 150 .

<AdultShape> a sh:NodeShape ;
   sh:subClassOf :Person ;
   sh:property <AdultAgeShape> .

<AdultAgeShape>
    sh:path :age;
    sh:minInclusive 19 ;
    sh:maxCount 1 ;
    sh:maxInclusive 150 .

<WomanShape> a sh:NodeShape ;
    sh:subClassOf :Adult ;
    sh:property <WomanAgeShape> .

<WomanAgeShape>
    sh:path :age;
    sh:targetClass <Woman> ;
    sh:minInclusive 19 ;
    sh:maxCount 1 ;
    sh:maxInclusive 150 .

<SeniorShape> a sh:NodeShape ;
   sh:subClassOf :Adult ;
   sh:property <SeniorAgeShape> .

<SeniorAgeShape>
    sh:path :age;
    sh:targetClass <Senior> ;
    sh:minInclusive 65 ;
    sh:maxCount 1 ;
    sh:maxInclusive 150 .

<ChildShape> a sh:NodeShape ;
   sh:subClassOf :Person ;
   sh:property <ChildAgeShape> .

<ChildAgeShape>
    sh:path :age;
    sh:targetClass <Child> ;
    sh:minInclusive 0 ;
    sh:maxCount 1 ;
    sh:maxInclusive 12 .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Beny ;
            sh:resultMessage "Value is not <= Literal(\"12\", datatype=xsd:integer)" ;
            sh:resultPath :age ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MaxInclusiveConstraintComponent ;
            sh:sourceShape :ChildAgeShape ;
            sh:value 13 ],
        [ a sh:ValidationResult ;
            sh:focusNode :John ;
            sh:resultMessage "Value is not <= Literal(\"150\", datatype=xsd:integer)" ;
            sh:resultPath :age ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MaxInclusiveConstraintComponent ;
            sh:sourceShape :PersonAgeShape ;
            sh:value 151 ],
        [ a sh:ValidationResult ;
            sh

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Lili,:age,-1,sh:MinInclusiveConstraintComponent,:PersonAgeShape,"Value is not >= Literal(""0"", datatype=xsd:inte..."
1,:John,:age,151,sh:MaxInclusiveConstraintComponent,:PersonAgeShape,"Value is not <= Literal(""150"", datatype=xsd:in..."
2,:Bibo,:age,None,sh:MaxCountConstraintComponent,:PersonAgeShape,More than 1 values on :Bibo->:age
3,:Peter,:age,None,sh:MinCountConstraintComponent,:PersonAgeShape,Less than 1 values on :Peter->:age
4,:Mary,:age,17,sh:MinInclusiveConstraintComponent,:WomanAgeShape,"Value is not >= Literal(""19"", datatype=xsd:int..."
5,:Senior2,:age,64,sh:MinInclusiveConstraintComponent,:SeniorAgeShape,"Value is not >= Literal(""65"", datatype=xsd:int..."
6,:Beny,:age,13,sh:MaxInclusiveConstraintComponent,:ChildAgeShape,"Value is not <= Literal(""12"", datatype=xsd:int..."


## Task 5
Jede Person hat maximal einen Namen. Der Name muss ein Literal sein und vom Datentyp String. Personen dürfen abgesehen von :name und rdf:type keine weiteren Eigenschaften haben.

In [7]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<John>  rdf:type  <MalePerson> ;
        :knows    <Mary> .

<FemalePerson>  rdfs:subClassOf  <Person> .

<Mary>  rdf:type  <FemalePerson> ;
        :name     "Mary" .

<Bibi>  rdf:type  <Person> ;
        :name     "Jim" , "Bibi" .

<Jane>  rdf:type  <Person> ;
        :name     2343 .

<Jim>   rdf:type  <Person> ;
        :name     <Jim> .

<MalePerson>  rdfs:subClassOf  <Person> .  
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<PersonShape> a sh:NodeShape ;
    sh:targetClass <Person> ;
    sh:property <PersonNameShape> ;
    sh:closed true ;
    sh:ignoredProperties ( rdf:type ) .

<PersonNameShape>
    sh:path :name;
    sh:maxCount 1 ;
    sh:datatype xsd:string ;
    sh:nodeKind sh:Literal .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Jim ;
            sh:resultMessage "Value is not of Node Kind sh:Literal" ;
            sh:resultPath :name ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeKindConstraintComponent ;
            sh:sourceShape :PersonNameShape ;
            sh:value :Jim ],
        [ a sh:ValidationResult ;
            sh:focusNode :Jane ;
            sh:resultMessage "Value is not Literal with datatype xsd:string" ;
            sh:resultPath :name ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:DatatypeConstraintComponent ;
            sh:sourceShape :PersonNameShape ;
            sh:value 2343 ],
        [ a sh:ValidationResult ;
            sh:focusNode :Bibi ;
        

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Jim,:name,:Jim,sh:DatatypeConstraintComponent,:PersonNameShape,Value is not Literal with datatype xsd:string
1,:Jim,:name,:Jim,sh:NodeKindConstraintComponent,:PersonNameShape,Value is not of Node Kind sh:Literal
2,:Bibi,:name,None,sh:MaxCountConstraintComponent,:PersonNameShape,More than 1 values on :Bibi->:name
3,:Jane,:name,2343,sh:DatatypeConstraintComponent,:PersonNameShape,Value is not Literal with datatype xsd:string
4,:John,:knows,:Mary,sh:ClosedConstraintComponent,:PersonShape,Node :John is closed. It cannot have value: :Mary


## Task 6
Städte sind via Property :inCountry Ländern zugeordnet. Europäische Städte sind europäischen Ländern zugeordnet. Österreichische Städte sind Austria zugeordnet.

In [8]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Paris>  rdf:type   <EuropeanCity> ;
        :inCountry  <France> .

<AustrianCity>  rdfs:subClassOf  <EuropeanCity> .

<Salzburg>  rdf:type  <AustrianCity> ;
        :inCountry  <Germany> .

<EuropeanCountry>  rdfs:subClassOf  <Country> .

<NewYork>  rdf:type  <City> ;
        :inCountry  <USA> .

<Germany>  rdf:type  <EuropeanCountry> .

<EuropeanCity>  rdfs:subClassOf  <City> .

<Vienna>  rdf:type  <AustrianCity> ;
        :inCountry  <Austria> .

<Austria>  rdf:type  <EuropeanCountry> .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<CountryShape> a sh:NodeShape ;
    sh:targetClass <Country> .

<CityShape> a sh:NodeShape ;
    sh:targetClass <City> ;
    sh:property <CountryInShape> .

<EuropeanCityShape> a sh:NodeShape ;
    sh:targetClass <EuropeanCity> ;
    sh:property <EuropeanCountryInShape> .

<AustrianCityShape> a sh:NodeShape ;
    sh:targetClass <AustrianCity> ;
    sh:property <AustrianCountryInShape> .

<CountryInShape>
    sh:path :inCountry ;
    sh:class <Country> .

<EuropeanCountryInShape>
    sh:path :inCountry ;
    sh:class <EuropeanCountry> .

<AustrianCountryInShape>
    sh:path :inCountry ;
    sh:in (<Austria>) .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :NewYork ;
            sh:resultMessage "Value does not have class :Country" ;
            sh:resultPath :inCountry ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :CountryInShape ;
            sh:value :USA ],
        [ a sh:ValidationResult ;
            sh:focusNode :Paris ;
            sh:resultMessage "Value does not have class :Country" ;
            sh:resultPath :inCountry ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :CountryInShape ;
            sh:value :France ],
        [ a sh:ValidationResult ;
            sh:focusNode :Paris ;
          

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Paris,:inCountry,:France,sh:ClassConstraintComponent,:CountryInShape,Value does not have class :Country
1,:NewYork,:inCountry,:USA,sh:ClassConstraintComponent,:CountryInShape,Value does not have class :Country
2,:Salzburg,:inCountry,:Germany,sh:InConstraintComponent,:AustrianCountryInShape,Value :Germany not in list [':Austria']
3,:Paris,:inCountry,:France,sh:ClassConstraintComponent,:EuropeanCountryInShape,Value does not have class :EuropeanCountry


## Task 7
Die Property :worksFor darf nur Personen (als Subjekt) und Organisation (als Objekt) miteinander verbinden.
Hinweis: Lösen Sie die Aufgabe ohne Property Shapes.


In [9]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Mary>  :worksFor  <JKU> .

<Jane>  rdf:type   <Person> ;
        :worksFor  <LMU> .

<Jim>   rdf:type   <Man> ;
        :worksFor  <LMU> .

<University>  rdfs:subClassOf  <Organization> .

<Bob>   rdf:type   <Person> ;
        :worksFor  [ rdf:type  <Organization> ] .

<JKU>   rdf:type  <University> . 
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<WorksForSubjectShape> a sh:NodeShape ;
    sh:class <Person> ;
    sh:targetSubjectsOf :worksFor .

<WorksForObjectShape> a sh:NodeShape ;
    sh:class <Organization> ;
    sh:targetObjectsOf :worksFor .
""")

# conforms, results_graph, results_text = shacl_validate(dg,sg)  

# if conforms:
# 	print("everything good")
# else:
# 	print(results_graph.serialize(format='turtle'))

# validation_report_as_dataframe(results_graph)

<Graph identifier=Nf9da1d17ad8e44cd8b752c921a3f526f (<class 'rdflib.graph.Graph'>)>

## Task 8
Peter kennt ausschließlich Männer, die in Österreich leben.

In [10]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<John>  rdf:type  <Man> ;
        :livesIn  <Austria> .

<Mary>  rdf:type  <Woman> ;
        :knows    <Jim> ;
        :livesIn  <Austria> .

<Hans>  rdf:type  <Man> .

<Peter>  :knows  <Josef> , <Hans> , <Franz> , <John> , <Mary> .

<Franz>  rdf:type  <Man> ;
        :livesIn  <Germany> .     
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<PeterShape> a sh:NodeShape ;
    sh:targetNode <Peter> ;
    sh:property <PeterKnowsShape> .

<PeterKnowsShape> a sh:PropertyShape ;
    sh:path :knows ;
    sh:node <AustrianManShape> .

<AustrianManShape> a sh:NodeShape ;
    sh:class <Man> ;
    sh:property <LivesInAustriaShape> .

<LivesInAustriaShape> a sh:PropertyShape ;
    sh:path :livesIn ;
    sh:in (<Austria>) ;
    sh:minCount 1 .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Peter ;
            sh:resultMessage "Value does not conform to Shape :AustrianManShape" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeConstraintComponent ;
            sh:sourceShape :PeterKnowsShape ;
            sh:value :Franz ],
        [ a sh:ValidationResult ;
            sh:focusNode :Peter ;
            sh:resultMessage "Value does not conform to Shape :AustrianManShape" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeConstraintComponent ;
            sh:sourceShape :PeterKnowsShape ;
            sh:value :Josef ],
        [ a sh:ValidationResult ;
            sh:focusNod

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Peter,:knows,:Hans,sh:NodeConstraintComponent,:PeterKnowsShape,Value does not conform to Shape :AustrianManShape
1,:Peter,:knows,:Franz,sh:NodeConstraintComponent,:PeterKnowsShape,Value does not conform to Shape :AustrianManShape
2,:Peter,:knows,:Josef,sh:NodeConstraintComponent,:PeterKnowsShape,Value does not conform to Shape :AustrianManShape
3,:Peter,:knows,:Mary,sh:NodeConstraintComponent,:PeterKnowsShape,Value does not conform to Shape :AustrianManShape


## Task 9
Das Nettogewicht eines Produkts darf nicht größer sein als das Bruttogewicht.
Verwenden Sie sh:lessThanOrEquals (siehe SHACL Recommendation).


In [11]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Cookies>  rdf:type   <Product> ;
        :grossWeight  0.2 ;
        :netWeight    0.12 .

<Milk>  rdf:type      <Product> ;
        :grossWeight  1.1 ;
        :netWeight    1 .

<Peter>  rdf:type     <Person> ;
        :grossWeight  74 ;
        :netWeight    72 .

<Bread>  rdf:type     <Product> ;
        :grossWeight  1.1 ;
        :netWeight    1.2 .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<ProductShape> a sh:NodeShape ;
   sh:targetClass <Product> ;
   sh:property <WeightShape> .

<WeightShape> a sh:PropertyShape ;
    sh:path :netWeight ;
    sh:lessThanOrEquals :grossWeight .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Bread ;
            sh:resultMessage "Value of :Bread->:grossWeight < Literal(\"1.2\", datatype=xsd:decimal)" ;
            sh:resultPath :netWeight ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:LessThanOrEqualsConstraintComponent ;
            sh:sourceShape :WeightShape ;
            sh:value 1.2 ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Bread,:netWeight,1.2,sh:LessThanOrEqualsConstraintComponent,:WeightShape,"Value of :Bread->:grossWeight < Literal(""1.2"",..."


## Task 10
Die Objekte von knows-Statements haben einen Namen (Property :name) oder haben einen Vornamen (:givenName) und einen Nachnamen (:familyName). eine

In [12]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Sara>  :givenName  "Sarah" .

<John>  :familyName  "Black" .

<Mary>  :knows  <Pete> .

<Bob>   :familyName  "Builder" ;
        :givenName   "Bob" .

<Pete>  :knows  <Bob> , <Sara> , <John> ;
        :name   "Peter Parker" .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<KnowsShape> a sh:PropertyNode;
    sh:targetObjectsOf :knows ;
    sh:or (
        [sh:and (
	   [
	      sh:path :familyName ;
	      sh:minCount 1 ;
	   ]
	   [
	      sh:path :givenName ;
	      sh:minCount 1 ;
	   ]
        )]
        [
           sh:path :name ;
	         sh:minCount 1 ;
        ]
    ) .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Sara ;
            sh:resultMessage "Node :Sara does not conform to one or more shapes in [ sh:and ( [ sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:path :familyName ] [ sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:path :givenName ] ) ] , [ sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:path :name ]" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:OrConstraintComponent ;
            sh:sourceShape :KnowsShape ;
            sh:value :Sara ],
        [ a sh:ValidationResult ;
            sh:focusNode :John ;
            sh:resultMessage "Node :John does not conform to one or more shapes in [ sh:and ( [ sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:path :familyName ] [ sh:

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:John,None,:John,sh:OrConstraintComponent,:KnowsShape,Node :John does not conform to one or more sha...
1,:Sara,None,:Sara,sh:OrConstraintComponent,:KnowsShape,Node :Sara does not conform to one or more sha...
